In [7]:
import pandas as pd

In [8]:
df =pd.read_csv("comparisons.csv", index_col=0)

In [9]:
df

,id,sentence,lang,llama,gpt4,gramm,group
0,1a,Which article did you throw away without reading,en,5,5,5,para-1
1,1a,Hvilken artikkel kastet du uten å lese,no,5,5,5,para-1
2,1b,Which article did you throw away without readi...,en,4,4,5,para-1
3,1b,Hvilken artikkel kastet du uten å lese den,no,5,5,5,para-1
4,1c,Which article did you throw it away without re...,en,2,1,1,para-1
...,...,...,...,...,...,...,...
57,rel7,Gutten som så filmen gikk hjem,no,5,5,5,yn-3
58,rel8,Has the boy who seen the movie will go home,en,1,1,1,yn-3
59,rel8,Så gutten som filmen gikk hjem,no,1,1,1,yn-3
60,rel9,Will the boy who has seen the movie go home,en,5,5,5,yn-3


## Overall

In [10]:
df_corrected = df

In [11]:
# Binarize model scores: 1–2 -> 1, 3–5 -> 5
df_binarized = df.copy()
df_binarized['llama_bin'] = df_binarized['llama'].apply(lambda x: 1 if x <= 2 else 5)
df_binarized['gpt4_bin'] = df_binarized['gpt4'].apply(lambda x: 1 if x <= 2 else 5)

# Compute match rate per language with binarized values
lang_summary_bin = df_binarized.groupby('lang').agg(
    N=('id', 'count'),
    LLama_avg=('llama', 'mean'),
    GPT4_avg=('gpt4', 'mean'),
    LLama_bin_match=('llama_bin', lambda x: (x == df_binarized.loc[x.index, 'gramm']).sum()),
    GPT4_bin_match=('gpt4_bin', lambda x: (x == df_binarized.loc[x.index, 'gramm']).sum())
)

lang_summary_bin['LLama_match_rate'] = (lang_summary_bin['LLama_bin_match'] / lang_summary_bin['N']).round(2)
lang_summary_bin['GPT4_match_rate'] = (lang_summary_bin['GPT4_bin_match'] / lang_summary_bin['N']).round(2)

lang_summary_bin = lang_summary_bin.drop(columns=['LLama_bin_match', 'GPT4_bin_match'])


In [12]:
lang_summary_bin

,N,LLama_avg,GPT4_avg,LLama_match_rate,GPT4_match_rate
lang,,,,,
en,31,3.451613,3.387097,0.94,0.97
no,31,3.870968,3.258065,0.55,0.77


In [13]:
# Binarize model scores: 1–2 -> 1, 3–5 -> 5

df_corrected['llama_bin'] = df_corrected['llama'].apply(lambda x: 1 if x <= 2 else 5)
df_corrected['gpt4_bin'] = df_corrected['gpt4'].apply(lambda x: 1 if x <= 2 else 5)

# Simplify group labels (e.g., atb-1, atb-2 → atb)
df_corrected['group_base'] = df_corrected['group'].str.extract(r'([a-z\-]+)')

# Group by language and simplified group
group_lang_summary_cleaned = df_corrected.groupby(['lang', 'group_base']).agg(
    N=('id', 'count'),
    LLama_avg=('llama', 'mean'),
    GPT4_avg=('gpt4', 'mean'),
    LLama_bin_match=('llama_bin', lambda x: (x == df_corrected.loc[x.index, 'gramm']).sum()),
    GPT4_bin_match=('gpt4_bin', lambda x: (x == df_corrected.loc[x.index, 'gramm']).sum())
)

# Add match rates
group_lang_summary_cleaned['LLama_match_rate'] = (
    group_lang_summary_cleaned['LLama_bin_match'] / group_lang_summary_cleaned['N']
).round(2)
group_lang_summary_cleaned['GPT4_match_rate'] = (
    group_lang_summary_cleaned['GPT4_bin_match'] / group_lang_summary_cleaned['N']
).round(2)

group_lang_summary_cleaned = group_lang_summary_cleaned.drop(columns=['LLama_bin_match', 'GPT4_bin_match'])

#display_dataframe_to_user("Cleaned Binarized Match Rates by Language and Group", group_lang_summary_cleaned)


In [14]:
group_lang_summary_cleaned

N  LLama_avg  GPT4_avg  LLama_match_rate  GPT4_match_rate
lang group_base                                                            
en   atb-         6   3.166667  3.833333              1.00             0.83
     lbe-         2   4.000000  2.000000              0.00             1.00
     para-       12   3.166667  2.916667              1.00             1.00
     rbe-         2   4.500000  5.000000              1.00             1.00
     yn-          9   3.666667  3.666667              1.00             1.00
no   atb-         7   4.571429  3.571429              0.57             0.57
     lbe-         2   2.000000  1.000000              0.00             0.00
     para-       11   4.818182  3.363636              0.55             1.00
     rbe-         2   2.000000  5.000000              0.00             1.00
     yn-          9   3.000000  3.000000              0.78             0.78